In [1]:
# !pip install hvplot panel tifffile numpy xarray scipy

In [2]:
import tifffile as tiff
import numpy as np
import hvplot.xarray
import panel as pn
import xarray as xr
import io
from scipy.ndimage import correlate

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [3]:
pn.extension()

This code creates an interactive interface using Panel widgets for uploading, playing, and summarizing `.tif` image files:

1. **File Input Widget**: `file_input` allows the user to upload a `.tif` file, restricting the accepted file types to `.tif` files and setting its width to 400 pixels.

2. **Frame Player**: `player` is a widget that enables the user to control and navigate through frames of the uploaded image, providing play, pause, and looping functionality. The player is initialized with a start and end value of 0, meaning it will be configured based on the uploaded image later.

3. **Summary Type Selector**: `summary_selector` allows the user to select a type of image summary, offering options such as 'Max Projection', 'Average', 'Min Projection', 'Local Correlation', and 'None'. This is used to apply different types of processing to the image data.

4. **Contrast Adjustment Variables**: `vmin` and `vmax` are initialized as placeholders for setting the minimum and maximum intensity values, which will be used for adjusting the contrast of the displayed image. 

In [4]:
file_input = pn.widgets.FileInput(accept='.tif', width=400)
player = pn.widgets.Player(name='Frame Player', start=0, end=0, value=0, loop_policy='loop', width=400)
summary_selector = pn.widgets.Select(name='Summary Type', options=['None', 'Max Projection', 'Average', 'Min Projection', 'Local Correlation'], width=200)
vmin, vmax = None, None


This function `calculate_local_correlation(da)` computes the local correlation of an input dataset, specifically for image or time-series data. Here’s what the code is doing:

1. **Kernel Creation**: A 3x3 averaging kernel (`kernel`) is created, where each element is equal to `1/9`, meaning it represents a local neighborhood averaging filter.

2. **Local Mean Calculation**: The function computes the mean across the frames (`dim='frame'`) of the input data array `da`. Then, it applies a 2D convolution (using `correlate`) with the averaging kernel to compute the local mean for each pixel or data point. The convolution uses the 'reflect' mode, which ensures that the edges are handled by reflecting the data beyond the boundary.

3. **Local Standard Deviation Calculation**: The function calculates the standard deviation of `da` along the frame axis (`axis=0`), then applies the same 2D convolution with the averaging kernel to compute the local standard deviation for each pixel or data point.

4. **Local Correlation**: Finally, the function calculates the local correlation by subtracting the local mean from the mean across frames and dividing by the local standard deviation. This results in a measure of how much the pixel's intensity deviates from the local mean in terms of standard deviation.

In [5]:
def calculate_local_correlation(da):
    kernel = np.ones((3, 3)) / 9.0
    local_mean = correlate(da.mean(dim='frame'), kernel, mode='reflect')
    local_std = correlate(np.std(da, axis=0), kernel, mode='reflect')
    return (da.mean(dim='frame') - local_mean) / local_std

This function `update_data_array(file)` processes the uploaded `.tif` file and updates the global variables `vmin` and `vmax`. Here's what each part of the function is doing:

1. **Global Variables**: The `vmin` and `vmax` variables are declared as global, meaning they are accessible and modifiable outside of the function.

2. **File Check**: The function checks if a file has been provided (`file is not None`) and if it has a non-zero length (`len(file) > 0`). This ensures that the function proceeds only if a valid file is uploaded.

3. **Reading the TIFF File**: 
   - The function reads the uploaded `.tif` file using `tiff.imread(io.BytesIO(file))`. The file is treated as a byte stream (`io.BytesIO(file)`), which is needed because the file comes from the FileInput widget.
   - This loads the TIFF image data into a NumPy array.

4. **Converting to xarray DataArray**: 
   - The image data is then converted into an xarray `DataArray` (`da`) with dimensions `["frame", "y", "x"]`, where:
     - `frame` refers to the different frames in the image (for multi-frame `.tif` files),
     - `y` and `x` represent the pixel coordinates for each frame.

5. **Updating `vmin` and `vmax`**: 
   - The function calculates the minimum and maximum pixel intensity values in the data array (`da.min()` and `da.max()`), converting them to Python scalars using `.item()`.
   - These values are stored in the global variables `vmin` and `vmax`, which are likely used elsewhere in the program for adjusting the display contrast.

6. **Return Value**: 
   - The function returns the `DataArray` (`da`) containing the image data. If no file is provided, the function returns `None`.

In [6]:
def update_data_array(file):
    global vmin, vmax
    if file is not None and len(file) > 0:
        tiff_data = tiff.imread(io.BytesIO(file))  # Read the TIFF data
        da = xr.DataArray(tiff_data, dims=["frame", "y", "x"])
        
        vmin = da.min().item()
        vmax = da.max().item()
        
        return da
    return None

This function `update_player(file)` is a dynamic callback function that depends on the `file_input` widget, meaning it automatically updates when a file is uploaded. Here’s what each part of the function does:

1. **`@pn.depends(file_input)`**: This decorator binds the function to the `file_input` widget, so the function is triggered whenever the file input changes (i.e., when a user uploads a file). The `file_input` widget is passed as an argument, allowing the function to react to the file changes.

2. **Calling `update_data_array(file_input.value)`**: 
   - The function retrieves the uploaded file’s data (`file_input.value`) and passes it to the `update_data_array()` function.
   - `update_data_array()` processes the `.tif` file (if it exists) and returns an xarray `DataArray` (`da`). If no file is uploaded or it's empty, `None` is returned.

3. **Player Widget Update**:
   - If `da` is not `None` (meaning a valid `.tif` file was uploaded and processed), the function updates the `player.end` property to the number of frames in the `DataArray` minus one (`da.sizes['frame'] - 1`). This ensures that the player widget's range is set correctly to allow navigation through all frames of the uploaded image.

4. **Return HTML Pane**:
   - If `da` is valid (file was uploaded and processed), the function returns a Panel `HTML` pane displaying a message indicating that the file was successfully uploaded, including the file name (`file_input.filename`).
   - If no file is uploaded or the file is invalid, the function returns a different `HTML` pane prompting the user to upload a `.tif` file.

In [7]:
@pn.depends(file_input)
def update_player(file):
    da = update_data_array(file_input.value)
    if da is not None:
        player.end = da.sizes['frame'] - 1  # Set the player range based on the number of frames in the file
    return pn.pane.HTML(f"<b>File Uploaded: {file_input.filename}</b>") if da is not None else pn.pane.HTML("<b>Upload a .tif file to display</b>")

This function `update_image(frame, file_input_value)` is designed to update the displayed image dynamically based on the current frame selected by the `player` widget and the `.tif` file uploaded by the `file_input` widget. Here’s how the function works:

1. **`@pn.depends(player, file_input)`**: This decorator makes the function dependent on two widgets: `player` (which controls the current frame) and `file_input` (which handles file uploads). The function will automatically update when either of these widgets' values change.

2. **Retrieving Data Array**:
   - The function calls `update_data_array(file_input_value)` to process the uploaded `.tif` file (or its byte stream). This returns an xarray `DataArray` (`da`) if the file is valid. If no file is uploaded or the file is empty, `da` will be `None`.
   
3. **Displaying the Selected Frame**:
   - If `da` is not `None` (i.e., a valid file is uploaded and processed), the function extracts the frame specified by the `player` widget (`da.isel(frame=frame)`), which selects the frame corresponding to the current position of the `player` (i.e., the frame number).
   - It then uses `hvplot.image()` to display the selected frame as an image. The following parameters are used:
     - `frame_height=400` and `frame_width=400`: These set the size of the image.
     - `cmap='viridis'`: The color map used for rendering the image.
     - `dynamic=True`: This ensures that the image is updated dynamically as the frame changes.
     - `clim=(vmin, vmax)`: The contrast limits for the image, using the global `vmin` and `vmax` values calculated from the data.

4. **Fallback for Missing File**:
   - If `da` is `None` (i.e., no file is uploaded), the function returns an HTML pane (`pn.pane.HTML`) with a message prompting the user to upload a `.tif` file.

In [8]:
@pn.depends(player, file_input)
def update_image(frame, file_input_value):
    da = update_data_array(file_input_value)
    if da is not None:
        return da.isel(frame=frame).hvplot.image(frame_height=400, frame_width=400, cmap='viridis', dynamic=True, clim=(vmin, vmax))
    return pn.pane.HTML("<b>Upload a .tif file to display</b>")

This function `display_summary(file_input_value, summary_type)` is designed to generate and display a summary of the uploaded `.tif` file based on the selected summary type. Here's how it works:

1. **`@pn.depends(file_input, summary_selector)`**: This decorator makes the function dependent on two widgets:
   - `file_input`: The widget that handles the uploading of `.tif` files.
   - `summary_selector`: The dropdown selector for choosing the type of summary to apply to the image data.

   The function will automatically update when either of these widgets changes.

2. **Retrieving Data Array**:
   - The function calls `update_data_array(file_input_value)` to process the uploaded `.tif` file and return an xarray `DataArray` (`da`). If no file is uploaded or if the file is invalid, `da` will be `None`.

3. **Checking Summary Type**:
   - If a valid file is uploaded (`da is not None`) and a summary type is selected (i.e., the `summary_type` is not `'None'`), the function proceeds to calculate the corresponding image summary based on the selected option in `summary_selector`.

4. **Summary Calculations**:
   - Depending on the selected summary type, the function performs the following operations:
     - **'Max Projection'**: It calculates the maximum projection of the frames using `da.max(dim='frame')`, which reduces the stack of frames by taking the maximum pixel value along the frame dimension.
     - **'Average'**: It calculates the average (mean) projection of the frames using `da.mean(dim='frame')`.
     - **'Min Projection'**: It calculates the minimum projection of the frames using `da.min(dim='frame')`.
     - **'Local Correlation'**: It calculates the local correlation using the custom function `calculate_local_correlation(da)`, which provides a correlation map across the image's local regions.

5. **Displaying the Summary**:
   - After computing the summary image based on the selected option, it is displayed using `hvplot.image()`. The image is rendered with the following settings:
     - `frame_height=400` and `frame_width=400`: Specifies the size of the image.
     - `cmap='viridis'`: Uses the 'viridis' color map for visualization.
     - `dynamic=True`: Ensures that the display updates dynamically based on the selection.

6. **Fallback for Missing File or No Summary**:
   - If no file is uploaded or if the summary type is `'None'`, the function returns an HTML pane with a message prompting the user to either upload a file or select a summary type.

In [9]:
@pn.depends(file_input, summary_selector)
def display_summary(file_input_value, summary_type):
    da = update_data_array(file_input_value)
    if da is not None and summary_type != 'None':
        if summary_type == 'Max Projection':
            summary_image = da.max(dim='frame')
        elif summary_type == 'Average':
            summary_image = da.mean(dim='frame')
        elif summary_type == 'Min Projection':
            summary_image = da.min(dim='frame')
        elif summary_type == 'Local Correlation':
            summary_image = calculate_local_correlation(da)
        
        return summary_image.hvplot.image(frame_height=400, frame_width=400, cmap='viridis', dynamic=True)
    return pn.pane.HTML("<b>Select a summary type or upload a .tif file</b>")

In [15]:
panel_layout = pn.Column(
    pn.pane.HTML("<h1 style='text-align:center; color:red;'>Calcium Imaging Data: Movie and Summary Images</h1>"),  
    pn.pane.Image("https://uni-bonn.sciebo.de/s/sGpK3h8C8g7Rujb/download", width=600),
    file_input, 
    update_player, 
    player, 
    update_image, 
    pn.Column(summary_selector, display_summary)
)
panel_layout.servable();

ValueError: Image pane cannot parse string that is not a filename or URL ('https://uni-bonn.sciebo.de/s/sGpK3h8C8g7Rujb/download').